# Imports

In [1]:
import os
import sys
import argparse
import numpy as np
import random
import pickle
import time

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader

from utils import getModelName, getNumberOfSpeakers
from data import Dataset

# Functions

# 1 - Input params

execute and copy

In [2]:
! python args_input_simulation.py

{'train_data_dir': '/scratch/speaker_databases/', 'valid_data_dir': '/scratch/speaker_databases/VoxCeleb-1/wav', 'train_labels_path': 'labels/Vox2.ndx', 'data_mode': 'normal', 'valid_clients': 'labels/clients.ndx', 'valid_impostors': 'labels/impostors.ndx', 'out_dir': './models/model1', 'model_name': 'CNN', 'front_end': 'VGG4L', 'window_size': 3.5, 'randomSlicing': False, 'normalization': 'cmn', 'kernel_size': 1024, 'embedding_size': 400, 'heads_number': 32, 'pooling_method': 'DoubleMHA', 'mask_prob': 0.3, 'scalingFactor': 30.0, 'marginFactor': 0.4, 'annealing': False, 'optimizer': 'Adam', 'learning_rate': 0.0001, 'weight_decay': 0.001, 'batch_size': 64, 'gradientAccumulation': 2, 'max_epochs': 1000000, 'early_stopping': 25, 'print_every': 1000, 'requeue': False, 'validate_every': 10000, 'num_workers': 2}


paste

In [3]:
params_dict = {'train_data_dir': '/scratch/speaker_databases/', 'valid_data_dir': '/scratch/speaker_databases/VoxCeleb-1/wav', 'train_labels_path': 'labels/Vox2.ndx', 'data_mode': 'normal', 'valid_clients': 'labels/clients.ndx', 'valid_impostors': 'labels/impostors.ndx', 'out_dir': './models/model1', 'model_name': 'CNN', 'front_end': 'VGG4L', 'window_size': 3.5, 'randomSlicing': False, 'normalization': 'cmn', 'kernel_size': 1024, 'embedding_size': 400, 'heads_number': 32, 'pooling_method': 'DoubleMHA', 'mask_prob': 0.3, 'scalingFactor': 30.0, 'marginFactor': 0.4, 'annealing': False, 'optimizer': 'Adam', 'learning_rate': 0.0001, 'weight_decay': 0.001, 'batch_size': 64, 'gradientAccumulation': 2, 'max_epochs': 1000000, 'early_stopping': 25, 'print_every': 1000, 'requeue': False, 'validate_every': 10000, 'num_workers': 2}

In [4]:
params = argparse.Namespace(**params_dict)

params.model_name = getModelName(params)
params.num_spkrs = getNumberOfSpeakers(params.train_labels_path) 

print(f"{params.num_spkrs} Speaker Labels")

if not os.path.exists(params.out_dir):
    os.makedirs(params.out_dir)
print(f"Output folder created at {params.out_dir}")
    
with open(params.out_dir + '/' + params.model_name + '_config.pkl', 'wb') as handle:
    pickle.dump(params, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"{params.out_dir + '/' + params.model_name + '_config.pkl'}")

5994 Speaker Labels
Output folder created at ./models/model1
./models/model1/CNN_VGG4L_3.5_128batchSize_0.0001lr_0.001weightDecay_1024kernel_400embSize_30.0s_0.4m_DoubleMHA_32_config.pkl


# 2 - Trainer class

In [5]:
class Trainer:

    def __init__(self, params, device):

        self.params = params
        self.device = device
        self.__load_data()
        
        
    def __load_data(self):
        print('Loading Data and Labels')
        with open(self.params.train_labels_path, 'r') as data_labels_file:
            train_labels = data_labels_file.readlines()

        data_loader_parameters = {
            'batch_size': self.params.batch_size, 
            'shuffle': True, 
            'num_workers': self.params.num_workers,
        }
        
        self.training_generator = DataLoader(Dataset(train_labels, self.params), **data_loader_parameters)

In [ ]:
torch.manual_seed(1234)
np.random.seed(1234)
    
#print('Defining Device...')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(f"device: {device}")
#print(torch.cuda.get_device_name(0))

trainer = Trainer(params, device)